In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [ ]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.52.3
Uninstalling transformers-4.52.3:
  Successfully uninstalled transformers-4.52.3
Found existing installation: accelerate 1.7.0
Uninstalling accelerate-1.7.0:
  Successfully uninstalled accelerate-1.7.0
  Using cached transformers-4.52.3-py3-none-any.whl.metadata (40 kB)
  Using cached accelerate-1.7.0-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.52.3-py3-none-any.whl (10.5 MB)
Using cached accelerate-1.7.0-py3-none-any.whl (362 kB)


In [ ]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
# from datasets import load_metric
import matplotlib.pyplot as plt
import pandas as pd

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [ ]:
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [ ]:
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineG

In [ ]:
# pip install -U datasets huggingface_hub fsspec

In [ ]:
from datasets import load_dataset

ds = load_dataset("allegro/summarization-allegro-articles")

train.csv:   0%|          | 0.00/108M [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/107M [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/8.73M [00:00<?, ?B/s]

dev.csv:   0%|          | 0.00/12.0M [00:00<?, ?B/s]

dev.csv:   0%|          | 0.00/11.8M [00:00<?, ?B/s]

...

test.csv:   0%|          | 0.00/29.7M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/2.43M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/73089 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8124 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/20304 [00:00<?, ? examples/s]

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['source', 'target'],
        num_rows: 73089
    })
    validation: Dataset({
        features: ['source', 'target'],
        num_rows: 8124
    })
    test: Dataset({
        features: ['source', 'target'],
        num_rows: 20304
    })
})

In [ ]:
ds["train"]["source"][0]

'Halka to dla wielu słowo nieznane. Niektórzy kojarzą je wprawdzie z częścią garderoby, nie wiedzą jednak, w jaki sposób się ją nosi. Komu i do czego służy halka? Halka brzmi niczym zdrobnienie od imienia Halina. Jest to również tytuł polskiej opery skomponowanej przez Stanisława Moniuszkę. Odnajdziemy ją także w branży odzieżowej. Oznacza część damskiej bielizny, którą kobiety zakładają pod suknię lub spódnicę. Jest dodatkiem do biustonosza i majtek. Wkłada się ją bezpośrednio pod suknię lub spódnicę. Poza tym halką nazywać można także koszule nocne, w których często śpią kobiety. Jakie rodzaje halek znajdziemy na rynku i czym powinniśmy się kierować przy ich zakupie?\nKlasyczna halka pod sukienkę\nCoraz rzadziej wkłada się halki pod codzienne stroje. Nasze babcie, a czasem nawet i mamy miewają to w zwyczaju do dziś. Zaglądając jednak do szaf nastolatek i młodych kobiet, ze znalezieniem halki można mieć duży problem. Istnieją bowiem alternatywne stroje, które mogą dać podobny efekt, c

In [ ]:
ds["train"]["target"][0]

'Halki – satynowe i bawełniane. Przegląd'

In [ ]:
split_lengths = [len(ds[split]) for split in ds]

print(f"Split Lengths: {split_lengths}")
print(f"Features: {ds['train'].column_names}")
print("\nSource:")

print(ds["test"]["source"][1])

print("\nTarget:")

print(ds["test"]["target"][1])

Split Lengths: [73089, 8124, 20304]
Features: ['source', 'target']

Source:
Chociaż pojemność pamięci smartfonów cały czas rośnie, to jednak wielu użytkowników potrzebuje w telefonie dodatkowej karty pamięci. W końcu na mobilnych urządzeniach trzymamy coraz więcej i więcej danych – muzykę, zdjęcia, a czasami nawet filmy. Tylko jaką kartę pamięci kupić? Wbrew pozorom kupno odpowiedniej karty pamięci do smartfona nie jest zadaniem łatwym. Z jednej strony do wyboru mamy tylko modele microSD, ale z drugiej koniecznie trzeba sprawdzić kilka parametrów.
Zanim kupisz kartę pamięci
Zanim postanowimy kupić kartę pamięci do smartfona, musimy sprawdzić kilka rzeczy, aby zakup nie okazał się bezcelowy. Przede wszystkim należy upewnić się, że nasz telefon w ogóle ma czytnik kart pamięci. Niestety, nie jest to standardem we wszystkich urządzeniach. Na przykład iPhone’y nigdy nie obsługiwały kart pamięci. Tak samo Nexusy (poza modelem Nexus One produkcji HTC). Karty pamięci nie włożymy także do Samsu

In [ ]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['source'], max_length=1024, truncation=True, padding='max_length')
    target_encodings = tokenizer(text_target=example_batch['target'], max_length=128, truncation=True, padding='max_length')

    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

ds_pt = ds.map(convert_examples_to_features, batched=True)


Map:   0%|          | 0/73089 [00:00<?, ? examples/s]

Map:   0%|          | 0/8124 [00:00<?, ? examples/s]

Map:   0%|          | 0/20304 [00:00<?, ? examples/s]

In [ ]:
ds_pt["train"]

Dataset({
    features: ['source', 'target', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 73089
})

In [ ]:
ds_pt["train"]["input_ids"][1]

[714,
 20638,
 6670,
 3033,
 1229,
 28064,
 2653,
 105,
 16927,
 116,
 12855,
 208,
 38416,
 10559,
 2653,
 12740,
 105,
 27411,
 167,
 1271,
 105,
 20063,
 26999,
 12855,
 32448,
 36059,
 144,
 110,
 7173,
 2103,
 40208,
 13291,
 107,
 52605,
 37409,
 2795,
 171,
 1332,
 12740,
 105,
 275,
 171,
 8874,
 15208,
 19717,
 33895,
 8232,
 1772,
 11577,
 108,
 22608,
 1191,
 105,
 252,
 12740,
 105,
 7936,
 110,
 45924,
 3027,
 12740,
 6046,
 110,
 34065,
 3957,
 22328,
 105,
 3033,
 17050,
 415,
 107,
 11101,
 316,
 415,
 110,
 80473,
 7710,
 554,
 532,
 80473,
 171,
 110,
 27943,
 4623,
 105,
 415,
 108,
 110,
 62187,
 84592,
 13687,
 12855,
 105,
 4429,
 19082,
 40208,
 1879,
 108,
 114,
 110,
 52145,
 16927,
 105,
 2795,
 108,
 114,
 1846,
 275,
 30807,
 46141,
 3225,
 105,
 108,
 15782,
 52888,
 49672,
 105,
 116,
 8996,
 151,
 20001,
 13075,
 4105,
 12855,
 532,
 13075,
 2278,
 105,
 1321,
 6046,
 2653,
 12740,
 105,
 108,
 20001,
 8232,
 1772,
 10455,
 3027,
 105,
 252,
 105,
 8874,


In [ ]:
ds_pt["train"]["attention_mask"][1]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [ ]:
# Training
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collector = DataCollatorForSeq2Seq(tokenizer, model = model_pegasus)

In [ ]:
!pip install -U transformers

In [ ]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir = 'pegasus-allegro',
    num_train_epochs = 1,
    warmup_steps = 500,
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    weight_decay = 0.01,
    logging_steps = 10,
    evaluation_strategy = 'steps',
    eval_steps = 500,
    save_steps = 1e6,
    gradient_accumulation_steps = 16
)

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
trainer = Trainer(model=model_pegasus,
                  args=trainer_args,
                  tokenizer=tokenizer,
                  data_collator=seq2seq_data_collator,
                  train_dataset=ds_pt["test"],
                  eval_dataset=ds_pt["validation"])

In [ ]:
trainer.train()

In [ ]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="source",
                               column_summary="target"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score


In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

In [ ]:
score = calculate_metric_on_test_ds(
    ds['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

In [ ]:
## Save model
model_pegasus.save_pretrained("pegasus-samsum-model")

In [ ]:
## Save tokenizer
tokenizer.save_pretrained("tokenizer")

In [ ]:
#Load
tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

In [ ]:
#Prediction
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}



sample_text = ds["test"][0]["source"]

reference = ds["test"][0]["target"]

pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)

##
print("Source:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])